# Data Preprocessing

In this sample application, we will explore image similarity search on Azure Cosmos DB for PostgreSQL using the [SemArt Dataset](https://researchdata.aston.ac.uk/id/eprint/380/). This dataset contains approximately 21k paintings gathered from the Web Gallery of Art. Each painting comes with various attributes, like a title, description, and the name of the artist. Let's embark on this creative exploration together!

In this notebook, we'll take the following steps:

* Load the data into a `pandas.DataFrame`.
* Clean up text descriptions by removing special characters and addressing other errors.
* Establish the dataset that will serve as the foundation for building the image similarity search application.

Before you start:

1. Download the dataset into the *semart_dataset* directory.
2. Create a [virtual environment](https://docs.python.org/3/library/venv.html) and activate it.
3. Install the [required Python packages](../requirements.txt).

In [1]:
import os
import pandas as pd
from unidecode import unidecode

In [2]:
dataset_folder = "semart_dataset"
filenames = [
    "semart_train.csv",
    "semart_test.csv",
    "semart_val.csv",
]

## Load the data into a `pandas.DataFrame`

The `load_data` function creates a DataFrame based on the filepaths provided for the CSV files.

In [3]:
def load_data(filepaths):
    return pd.concat((pd.read_csv(file, sep="\t", encoding="cp1252") for file in filepaths))

In [4]:
filepaths = [os.path.join("..", dataset_folder, filename) for filename in filenames]
data = load_data(filepaths)

Let's explore the dataset!

In [5]:
data.head(10)

,IMAGE_FILE,DESCRIPTION,AUTHOR,TITLE,TECHNIQUE,DATE,TYPE,SCHOOL,TIMEFRAME
0,19873-1darmst.jpg,"The Meyer or Darmstadt Madonna is the last, mo...","HOLBEIN, Hans the Younger",Darmstadt Madonna,"Oil on limewood, 147 x 102 cm",1526 and after 1528,religious,German,1501-1550
1,18759-guard301.jpg,Whereas Canaletto incorporates classically ins...,"GUARDI, Francesco",Landscape with a Fisherman's Tent,"Oil on canvas, 49 x 77 cm",1770-75,landscape,Italian,1751-1800
2,04589-temptati.jpg,In this painting the refinement of the colouri...,"BILIVERT, Giovanni",The Temptation of Charles and Ubalde,"Oil on copper, 37 x 28 cm",1629-30,religious,Italian,1601-1650
3,15104-magi.jpg,"The left side of the painting was cut, origina...",GEERTGEN tot Sint Jans,Adoration of the Magi,"Panel, 111 x 69 cm",1480-85,religious,Netherlandish,1451-1500
4,36582-paolo_f1.jpg,The painting illustrates a famous episode from...,"SCHEFFER, Ary",The Ghosts of Paolo and Francesca Appear to Da...,"Oil on canvas, 167 x 234 cm",1835,other,Dutch,1801-1850
5,00930-30coron.jpg,Fra Angelico dedicated several of his works to...,"ANGELICO, Fra",The Coronation of the Virgin,"Tempera on wood, 112 x 114 cm",1434-35,religious,Italian,1401-1450
6,24485-01view.jpg,The Bardi di Vernio Chapel in Santa Croce is o...,MASO DI BANCO,View of the Bardi di Vernio Chapel from the So...,Fresco,c. 1335,religious,Italian,1301-1350
7,18424-2105grec.jpg,This portrait is an example of El Greco's late...,"GRECO, El",Portrait of Cardinal Tavera,"Oil on canvas, 103 x 83 cm",1608-14,portrait,Spanish,1551-1600
8,21949-4lastsu3.jpg,Leonardo's painting of the Last Supper was con...,LEONARDO da Vinci,The Last Supper,"Mixed technique, 460 x 880 cm",1498,religious,Italian,1451-1500
9,03093-calvary.jpg,Traditionally the painting was ascribed to Pie...,"BALTENS, Peeter",Christ on the Road to Calvary,"Oil on oak panel, 116 x 156 cm",1560s,religious,Flemish,1501-1550


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21382 entries, 0 to 1068
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   IMAGE_FILE   21382 non-null  object
 1   DESCRIPTION  21382 non-null  object
 2   AUTHOR       21382 non-null  object
 3   TITLE        21382 non-null  object
 4   TECHNIQUE    21382 non-null  object
 5   DATE         21382 non-null  object
 6   TYPE         21382 non-null  object
 7   SCHOOL       21382 non-null  object
 8   TIMEFRAME    21382 non-null  object
dtypes: object(9)
memory usage: 1.6+ MB


In [7]:
print(f"Dataframe shape: {data.shape}")

Dataframe shape: (21382, 9)


## Data Processing

Initially, make a copy of the original data and eliminate any `NaN` values and duplicate image files.

In [8]:
df = data.dropna()
df.drop_duplicates(subset=["IMAGE_FILE"], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21382 entries, 0 to 1068
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   IMAGE_FILE   21382 non-null  object
 1   DESCRIPTION  21382 non-null  object
 2   AUTHOR       21382 non-null  object
 3   TITLE        21382 non-null  object
 4   TECHNIQUE    21382 non-null  object
 5   DATE         21382 non-null  object
 6   TYPE         21382 non-null  object
 7   SCHOOL       21382 non-null  object
 8   TIMEFRAME    21382 non-null  object
dtypes: object(9)
memory usage: 1.6+ MB


Since the `DATE` and `SCHOOL` information won't be used in the image similarity search app, you can remove them from the DataFrame.

In [9]:
df = df.drop(columns=["DATE", "SCHOOL"])
df.head()

,IMAGE_FILE,DESCRIPTION,AUTHOR,TITLE,TECHNIQUE,TYPE,TIMEFRAME
0,19873-1darmst.jpg,"The Meyer or Darmstadt Madonna is the last, mo...","HOLBEIN, Hans the Younger",Darmstadt Madonna,"Oil on limewood, 147 x 102 cm",religious,1501-1550
1,18759-guard301.jpg,Whereas Canaletto incorporates classically ins...,"GUARDI, Francesco",Landscape with a Fisherman's Tent,"Oil on canvas, 49 x 77 cm",landscape,1751-1800
2,04589-temptati.jpg,In this painting the refinement of the colouri...,"BILIVERT, Giovanni",The Temptation of Charles and Ubalde,"Oil on copper, 37 x 28 cm",religious,1601-1650
3,15104-magi.jpg,"The left side of the painting was cut, origina...",GEERTGEN tot Sint Jans,Adoration of the Magi,"Panel, 111 x 69 cm",religious,1451-1500
4,36582-paolo_f1.jpg,The painting illustrates a famous episode from...,"SCHEFFER, Ary",The Ghosts of Paolo and Francesca Appear to Da...,"Oil on canvas, 167 x 234 cm",other,1801-1850


### Convert column names to lowercase.

In [10]:
df.columns = map(str.lower, df.columns)
df.head()

,image_file,description,author,title,technique,type,timeframe
0,19873-1darmst.jpg,"The Meyer or Darmstadt Madonna is the last, mo...","HOLBEIN, Hans the Younger",Darmstadt Madonna,"Oil on limewood, 147 x 102 cm",religious,1501-1550
1,18759-guard301.jpg,Whereas Canaletto incorporates classically ins...,"GUARDI, Francesco",Landscape with a Fisherman's Tent,"Oil on canvas, 49 x 77 cm",landscape,1751-1800
2,04589-temptati.jpg,In this painting the refinement of the colouri...,"BILIVERT, Giovanni",The Temptation of Charles and Ubalde,"Oil on copper, 37 x 28 cm",religious,1601-1650
3,15104-magi.jpg,"The left side of the painting was cut, origina...",GEERTGEN tot Sint Jans,Adoration of the Magi,"Panel, 111 x 69 cm",religious,1451-1500
4,36582-paolo_f1.jpg,The painting illustrates a famous episode from...,"SCHEFFER, Ary",The Ghosts of Paolo and Francesca Appear to Da...,"Oil on canvas, 167 x 234 cm",other,1801-1850


### Transliterate non-ASCII characters

The dataset includes various non-ASCII characters like *ú*, *é*, *î*, and *ä*. In this section, the aim is to convert these non-ASCII characters to their ASCII equivalents, minimizing potential issues associated with character encoding.

The `find_non_ascii_characters` function returns a list of all non-ASCII characters present in the dataset. You will then use the `unidecode` function to convert these special characters into their ASCII representation.

In [11]:
def find_non_ascii_characters(dataframe):
    non_ascii_characters = set()

    # Iterate through each cell in the dataframe
    for col in dataframe.columns:
        for cell in dataframe[col].values:
            # Check for non-ASCII characters and add them to the set
            non_ascii_characters.update(char for char in cell if ord(char) > 127)

    return list(non_ascii_characters)

In [12]:
non_ascii_characters_list = find_non_ascii_characters(df)
print("List of non-ASCII characters:", non_ascii_characters_list)

List of non-ASCII characters: ['í', 'Ó', 'ú', 'ë', 'Â', '’', 'Á', 'Ô', 'ö', 'é', 'î', 'ü', 'ó', 'Ë', 'è', 'â', 'ä', '×', 'ô', 'É', 'ç', 'Í', 'ž', 'Ü', 'á', 'Ö', 'Ú', 'Ä', 'Ç', 'õ', 'û']


In [13]:
df = df.map(lambda x: unidecode(x))

In [14]:
non_ascii_characters_list = find_non_ascii_characters(df)
print("List of non-ASCII characters:", non_ascii_characters_list)

List of non-ASCII characters: []


### Exclude the painting size information from the `technique` column

Let's remove the painting size information from the `technique` column since it won't be utilized in our app.

In [15]:
df["technique"] = df["technique"].apply(lambda x: x.split(sep=", ")[0])
df.head()

,image_file,description,author,title,technique,type,timeframe
0,19873-1darmst.jpg,"The Meyer or Darmstadt Madonna is the last, mo...","HOLBEIN, Hans the Younger",Darmstadt Madonna,Oil on limewood,religious,1501-1550
1,18759-guard301.jpg,Whereas Canaletto incorporates classically ins...,"GUARDI, Francesco",Landscape with a Fisherman's Tent,Oil on canvas,landscape,1751-1800
2,04589-temptati.jpg,In this painting the refinement of the colouri...,"BILIVERT, Giovanni",The Temptation of Charles and Ubalde,Oil on copper,religious,1601-1650
3,15104-magi.jpg,"The left side of the painting was cut, origina...",GEERTGEN tot Sint Jans,Adoration of the Magi,Panel,religious,1451-1500
4,36582-paolo_f1.jpg,The painting illustrates a famous episode from...,"SCHEFFER, Ary",The Ghosts of Paolo and Francesca Appear to Da...,Oil on canvas,other,1801-1850


### Specify the data type of each column as `string`

In [16]:
df = df.convert_dtypes()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21382 entries, 0 to 1068
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_file   21382 non-null  string
 1   description  21382 non-null  string
 2   author       21382 non-null  string
 3   title        21382 non-null  string
 4   technique    21382 non-null  string
 5   type         21382 non-null  string
 6   timeframe    21382 non-null  string
dtypes: string(7)
memory usage: 1.3 MB


### Exclude artists who have fewer than 15 paintings from the dataset

In [17]:
df.value_counts("author")

author
GOGH, Vincent van                 327
REMBRANDT Harmenszoon van Rijn    262
GIOTTO di Bondone                 249
RUBENS, Peter Paul                213
TIZIANO Vecellio                  205
                                 ... 
ZUCCHI, Francesco                   1
ZIMMERMANN, Carl Friedrich          1
ZIMMERMANN, August Albert           1
AENVANCK, Theodoor                  1
ADLER, Salomon                      1
Name: count, Length: 3281, dtype: int64

In [18]:
MIN_NUMBER_OF_PAINTINGS_PER_ARTIST = 15
df = df[df.groupby("author").author.transform("count") > MIN_NUMBER_OF_PAINTINGS_PER_ARTIST]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11631 entries, 0 to 1067
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_file   11631 non-null  string
 1   description  11631 non-null  string
 2   author       11631 non-null  string
 3   title        11631 non-null  string
 4   technique    11631 non-null  string
 5   type         11631 non-null  string
 6   timeframe    11631 non-null  string
dtypes: string(7)
memory usage: 726.9 KB


## Address any remaining issues in the dataset

### Refine the names of the artists in the dataset

Several authors' names contain a *?* in place of non-ASCII characters. The `find_questionmarks_in_author_name` function provides a list of these incorrect artist names. Subsequently, you will manually correct these names in the dataset, considering that the *?* represents a different character in each case.

In [19]:
def find_questionmarks_in_author_name(dataframe):
    authors = set()
    
    # Iterate through each cell in the dataframe
    for cell in dataframe["author"].values:
        # Check for ? and add the name to the set
        if any([True for char in cell if char == "?"]):
            authors.add(cell)
    
    return list(authors)

In [20]:
authors = find_questionmarks_in_author_name(df)
print(f"Number of authors with incorrect decoded name: {len(authors)}")
for author in authors:
    print(author)

Number of authors with incorrect decoded name: 2
TURA, Cosm?
DELACROIX, Eug?ne


In [21]:
REPLACEMENTS = [
    ("TURA, Cosm?", "TURA, Cosme"),
    ("DELACROIX, Eug?ne", "DELACROIX, Eugene"),
]
for incorrect_name, correct_name in REPLACEMENTS:
    df["author"] = df["author"].apply(lambda x: x.replace(incorrect_name, correct_name))

### Exclude Unknown Authors

For paintings where the artist is unidentified, the artist's name is presented as: *UNKNOWN MASTER, \<SCHOOL\>*. All entries with this information will be removed.

In [22]:
df = df[~df["author"].str.contains("UNKNOWN")]

## Save the final dataset

Let's look at some data for the revised dataset.

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11206 entries, 0 to 1067
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_file   11206 non-null  string
 1   description  11206 non-null  string
 2   author       11206 non-null  object
 3   title        11206 non-null  string
 4   technique    11206 non-null  string
 5   type         11206 non-null  string
 6   timeframe    11206 non-null  string
dtypes: object(1), string(6)
memory usage: 700.4+ KB


In [24]:
df.nunique()

image_file     11206
description    10503
author           250
title           8238
technique        339
type              10
timeframe         21
dtype: int64

Save the clean dataset as a `csv` file in the *dataset* drectory.

In [25]:
final_dataset_folder = os.path.join("..", "dataset")
dataset_csv_filename = "dataset.csv"

os.makedirs(final_dataset_folder, exist_ok=True)  
df.to_csv(
    os.path.join(final_dataset_folder, dataset_csv_filename),
    index=False,
    sep="\t",
)